# Station BSG - SpecUFEx workflow 

In [1]:

###################################
##   I. Import waveforms, save to H5
##  II. Convert to spectrograms, save to H5
## III. Run specufex, save fingerprints to H5
###################################


import pandas as pd
import numpy as np
import obspy
import os
import sys
import h5py
import yaml
import sys
sys.path.append('src/')
import glob
import datetime
from f1_spectrogram_functions import wf_to_H5, gen_sgram_QC_noAlias

sys.path.append('./specufex/specufex/')
from specufex import BayesianNonparametricNMF, BayesianHMM




In [2]:
## From previous notebook
cat_region = pd.read_csv('../data/catalogs/NCADDiff_5km_2019_500mFault_WS21.csv')
evID_region = cat_region.event_ID


In [3]:


###################################
## Build paths and load settings
###################################


plot = 0   # plot spectrograms?
mod = 2000 # Print 1 spectrogram for every %mod spectrograms generated 

yamlPath = "./SA_REQS_BSG.yaml"
with open(yamlPath) as stream:
    config = yaml.safe_load(stream)
    
path_config = config["paths"]
key = path_config["key"]
data_config = config['dataParams']
station = data_config["station"]
channel = data_config["channel"]
channel_ID = data_config["channel_ID"]
sampling_rate = data_config["sampling_rate"]

# build path strings
dataH5_name = f'data_{key}.h5'
projectPath = path_config["projectPath"]
pathCatFold = path_config["pathCatFold"]


# pathWF = path_config["pathWF"]
pathWF = '/Users/theresasawi/Documents/11_Manuscripts/SA_REQS/data/raw/BSG/'


wf_cat_out_path = projectPath + f'{key}_wf_cat_out.csv'

if not os.path.isdir(projectPath + 'data/H5files/'):
    os.mkdir(projectPath + 'data/H5files/')
    
dataH5_path = projectPath + 'data/H5files/' + dataH5_name
    

In [4]:


################################################
## Create catalog with event ids made from filenames 
################################################
wf_list = glob.glob(pathWF + '*')

filenames = [path.split('/')[-1] for path in wf_list]
    

ev_ID = [int(path.split('/')[-1].split('.')[-1]) for path in wf_list]

print('Example evID: ', ev_ID[0])
cat_paths = pd.DataFrame({"ev_ID":ev_ID,
                          "event_ID":ev_ID,
                          "filename":filenames})


wf_test = obspy.read(pathWF + cat_paths.filename.iloc[0])

lenData = len(wf_test[channel_ID])
sr = fs = wf_test[channel_ID].stats.sampling_rate

print(sr, " Hz; ", lenData, " samples", len(filenames), 'waveforms')

Example evID:  72354471
100.0  Hz;  2000  samples 51442 waveforms


In [5]:
cat_paths_region = cat_paths[cat_paths.event_ID.isin(evID_region)]

In [6]:

###################################
## Set spectrogram parameters
###################################

sgram_config = config["sgramParams"]
nfft = sgram_config["nfft"]
fmin, fmax = sgram_config["fmin"], sgram_config["fmax"]
tmin, tmax = sgram_config["tmin"], sgram_config["tmax"]

SpecUFEx_H5_name = 'SpecUFEx_' + path_config["h5name"] #f'SpecUFEx_{key}.hdf5'
SpecUFEx_H5_path = projectPath + 'data/H5files/' + SpecUFEx_H5_name


##spectrogram parameters, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html
nperseg = int(sgram_config["winLen_Sec"]*fs) #datapoints per window segment
noverlap = int(nperseg*sgram_config["fracOverlap"])  #fraction of window overlapped  

#padding must be longer than n per window segment
if nfft < nperseg:
    nfft = nperseg*2
    print("nfft too short; changing to ", nfft)

mode='magnitude'
scaling='spectrum'

# set args for generator
args = {'station':station,
        'channel':channel,
        'fs': fs,
        'lenData': lenData,
        'nperseg': nperseg,
        'noverlap': noverlap,
        'nfft': nfft,
        'mode': mode,
        'scaling': scaling,
        'fmin': fmin,
        'fmax': fmax,
        'tmin':tmin,
        'tmax':tmax}

In [7]:
###################################
## Save waveforms to H5
###################################

evID_keep, wf_example = wf_to_H5(projectPath,dataH5_path,pathWF,cat_paths_region,lenData,channel_ID,station,channel,t0=0,t1=10000000000, verbose=0)



17000 / 3640
0  duplicate events found and avoided
3637  waveforms loaded


In [8]:
len(evID_keep)

3637

In [9]:
dataH5_path

'../data/H5files/data_SA_REQS_BSG.h5'

# Save processing information

In [10]:
###################################
## Save processing information to data H5
###################################
with h5py.File(dataH5_path,'a') as h5file:
    processing_group = h5file.create_group(f"{station}/processing_info")
    processing_group.create_dataset(name= "sampling_rate_Hz", data=sampling_rate)#,dtype='S')
    # processing_group.create_dataset(name= "station_info", data=station_info)
    # processing_group.create_dataset(name= "calibration", data=calib)#,dtype='S')
    # processing_group.create_dataset(name= "orig_formata", data=_format)#,dtype='S')
    # processing_group.create_dataset(name= "instr_response", data=instr_response,dtype='S')
    processing_group.create_dataset(name= "lenData", data=lenData)#,dtype='S')
    


In [11]:

######################################################
## Instantiate generator and generate spectrograms
######################################################


trim = True

# put sgrams in h5
gen_sgram = gen_sgram_QC_noAlias(5,key,
                        evID_list=evID_keep,
                        dataH5_path = dataH5_path,#h5 data file
                        h5File=SpecUFEx_H5_path, #h5 sgram file
                        trim=trim, #trim to min and max freq
                        saveMat=False, #set true to save folder of .mat files
                        sgramOutfile='.', #path to save .mat files
                        **args
                        ) #path to save sgram figures





######################################################
## Part 2: Make spectrograms
######################################################
    
evID_list_QC_sgram = []
spectra_for_avg = []

less10=0
with h5py.File(SpecUFEx_H5_path,'a') as fileLoad:

    n=0
    Nkept=0

    if 'spectrograms' in fileLoad.keys():
        del fileLoad["spectrograms"]

    if 'sgram_normConst' in fileLoad.keys():
        del fileLoad["sgram_normConst"]

    spectrograms_group     = fileLoad.create_group(f"spectrograms")

    sgram_normConst_group  = fileLoad.create_group(f"sgram_normConst")

    lenEv = len(evID_keep)
    
    while n <= lenEv: ## not sure a better way to execute this? But it works
        try:   #catch generator "stop iteration" error
            evID,sgram,fSTFT,tSTFT, normConstant, Nkept,evID_BADones, i = next(gen_sgram) #next() command updates generator
             
            n = i+1
            evID = str(evID)
            

            if not evID in spectrograms_group:
                
                if np.max(sgram)>10: #v8 values below this were typically from bad stations 
                    
                    spectrograms_group.create_dataset(name= evID, data=sgram)
                    evID_list_QC_sgram.append(np.int64(evID))
                    spectra_for_avg.append(np.array(sgram))
                    
                    
                    
                    if not evID in sgram_normConst_group:
                        
                        sgram_normConst_group.create_dataset(name= evID, data=normConstant)
#                 else:
#                     print("dB less than 20")
                    

                    if plot:

                        if n%mod==0:
                            plt.figure()
                            plt.pcolormesh(tSTFT,fSTFT,sgram,shading='auto')
                            cbar = plt.colorbar(pad=.06)
                            cbar.set_label('dB',labelpad=8)#,fontsize = 14)
                        #     plt.clim(0,45)
                            plt.xlabel('time (s)')
                            plt.ylabel('frequency (Hz)')
                            plt.show()
                else:
                    less10 +=1
                    

        except StopIteration: #handle generator error
            break

    print('N events in evID_list_QC_sgram:', len(evID_list_QC_sgram))
    print('N events in evID_BADones:', len(evID_BADones))

    if 'spec_parameters' in fileLoad.keys():
        del fileLoad["spec_parameters"]
        

cat_final = cat_paths[cat_paths.event_ID.isin(evID_list_QC_sgram)]
cat_final.to_csv(pathCatFold + f'{key}_cat_keep_sgram.csv')


0 / 3637
1000 / 3637
2000 / 3637
3000 / 3637
N events in evID_list_QC_sgram: 3637
N events in evID_BADones: 0


In [12]:

###################################
## Save processing information to spectrogram H5
###################################
with h5py.File(SpecUFEx_H5_path,'a') as fileLoad:
    
    spec_parameters_group  = fileLoad.create_group(f"spec_parameters")
    
    spec_parameters_group.clear()

    spec_parameters_group.create_dataset(name= 'fs', data=fs)
    spec_parameters_group.create_dataset(name= 'lenData', data=lenData)
    spec_parameters_group.create_dataset(name= 'nperseg', data=nperseg)
    spec_parameters_group.create_dataset(name= 'noverlap', data=noverlap)
    spec_parameters_group.create_dataset(name= 'nfft', data=nfft)
    spec_parameters_group.create_dataset(name= 'mode', data=mode)
    spec_parameters_group.create_dataset(name= 'scaling', data=scaling)
    spec_parameters_group.create_dataset(name= 'fmin', data=fmin)
    spec_parameters_group.create_dataset(name= 'fmax', data=fmax)
    spec_parameters_group.create_dataset(name= 'tmin', data=tmin)
    spec_parameters_group.create_dataset(name= 'tmax', data=tmax)    
    spec_parameters_group.create_dataset(name= 'fSTFT', data=fSTFT)
    spec_parameters_group.create_dataset(name= 'tSTFT', data=tSTFT)    


In [13]:
len(evID_list_QC_sgram)

3637

# Part 3: SpecUFEx

### Linearize spectrogram

In [14]:
import os
import time

starttime = time.time()

X = []

with h5py.File(SpecUFEx_H5_path,'a') as fileLoad:
    for evID in fileLoad['spectrograms']:
        specMat = fileLoad['spectrograms'].get(evID)[:]
        X.append(specMat)

    X = np.array(X)

# ================
print(np.shape(X))


# print(X[:,:,-1])

# # IOPub data rate exceeded.
# # The notebook server will temporarily stop sending output
# # to the client in order to avoid crashing it.
# # To change this limit, set the config variable
# # `--NotebookApp.iopub_data_rate_limit`.

# # Current values:
# # NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
# # NotebookApp.rate_limit_window=3.0 (secs)

specparams = config["specufexParams"]

print(specparams["nmf_batchsz"],specparams["hmm_batchsz"])




#%% ============================================================
# Running SpecUFEx
#%% ============================================================

specparams = config["specufexParams"]

print('Running NMF')
nmf = BayesianNonparametricNMF(X.shape)
for i in range(specparams["nmf_nbatch"]):
    # pick random sample
    print(f"Batch {i}")
#     sample = np.random.choice(X.shape[0], specparams["nmf_batchsz"])
#     nmf.fit(X[sample], verbose=0)
    nmf.fit(X, verbose=0)

print('transforming NMF')    
Vs = nmf.transform(X)
# print how long it took

#%%
print('Running HMM')
hmm = BayesianHMM(nmf.num_pat, nmf.gain)
for i in range(specparams["hmm_nbatch"]):
    print(f"Batch {i}")
#     sample = np.random.choice(Vs.shape[0], specparams["nmf_batchsz"])
    hmm.fit(Vs)

print('transforming HMM')    
fingerprints, As, gams = hmm.transform(Vs)

# print(fingerprints[0])

# show a fingerprint if you want to .. but not useful for running remotely..
# plt.imshow(fingerprints[0])
# plt.show()
#%%

endtime = time.time()

time_elapsed = endtime - starttime


os.system("say 'SpecUFEx complete'")  
print(time_elapsed, ' s elapsed')


(3637, 308, 200)
100000 100000
Running NMF
Batch 0
transforming NMF
Running HMM
Batch 0
transforming HMM
1893.879991054535  s elapsed


In [15]:

# =============================================================================
# save output to H5
# =============================================================================
print('writing all output to h5')
with h5py.File(SpecUFEx_H5_path,'a') as fileLoad:


    ##fingerprints are top folder
    if 'fingerprints' in fileLoad.keys():
        del fileLoad["fingerprints"]
    fp_group = fileLoad.create_group('fingerprints')

    if 'SpecUFEX_output' in fileLoad.keys():
        del fileLoad["SpecUFEX_output"]
    out_group = fileLoad.create_group("SpecUFEX_output")

    # write fingerprints: ===============================
    for i, evID in enumerate(fileLoad['spectrograms']):
        fp_group.create_dataset(name= evID, data=fingerprints[i])


    # write the SpecUFEx out: ===========================
    # maybe include these, but they are not yet tested.
    ACM_group = fileLoad.create_group("SpecUFEX_output/ACM")
    STM_group = fileLoad.create_group("SpecUFEX_output/STM")

    for i, evID in enumerate(fileLoad['spectrograms']):
        ACM_group.create_dataset(name=evID,data=Vs[i]) #ACM
        STM_group.create_dataset(name=evID,data=gams[i]) #STM

    gain_group = fileLoad.create_group("SpecUFEX_output/ACM_gain")
    W_group                      = fileLoad.create_group("SpecUFEX_output/W")
    EB_group                     = fileLoad.create_group("SpecUFEX_output/EB")
    ## # # delete probably ! gain_group                   = fileLoad.create_group("SpecUFEX_output/gain")
    #RMM_group                    = fileLoad.create_group("SpecUFEX_output/RMM")

    W_group.create_dataset(name='W',data=nmf.EW)
    EB_group.create_dataset(name=evID,data=hmm.EB)
    gain_group.create_dataset(name='gain',data=nmf.gain) #same for all data
    # RMM_group.create_dataset(name=evID,data=RMM)
    

    

writing all output to h5
